# Aspect sentiment analysis (Topic Modeling)
### Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

### Goal is Social Listening
* Basically it is finding Topic on which Amazon India customer are talking about on Social Media.
* social media is a platform where people are vocal regarding there Fact and Opinion.
* Whether it is positive or negative, they express there view.
* Its alway hard to express the view in person, so there is always the bais involed to certain degree.
* But on Internet there is no real consequence. so it makes easy for customer.
* There is even more psychology reason behind.

## Amazon India

In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk

In [2]:
# read the data in
df = pd.read_csv(r"C:\Users\ASUS\abhi\Twitter sentiment analysis\amazon.csv")

In [3]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [4]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(df.text.apply(round1))
data_clean

,text
0,indiaprogress kinda wrong world runs on amazon...
1,jeffbezos im from india sir your taken my mo...
2,before making your goals for the year read m...
3,ola to set up worlds largest escooter factory ...
4,amazonin \nis allowed to sell made in chinese ...
...,...
3095,amazon echo gen echo dot gen echo dot gen w...
3096,deal price ₹ \ndiscount ₹
3097,deal price ₹ \ndiscount ₹
3098,amazon prime video’s ‘watch party’ now in indi...


In [5]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('[₹]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [6]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.text.apply(round2))
data_clean

,text
0,indiaprogress kinda wrong world runs on amazon...
1,jeffbezos im from india sir your taken my mo...
2,before making your goals for the year read m...
3,ola to set up worlds largest escooter factory ...
4,amazonin is allowed to sell made in chinese mo...
...,...
3095,amazon echo gen echo dot gen echo dot gen w...
3096,deal price discount
3097,deal price discount
3098,amazon prime videos watch party now in india h...


In [7]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

In [9]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [10]:
# Apply the nouns function to the text to filter only on nouns
data_nouns = pd.DataFrame(data_clean.text.apply(nouns))
data_nouns

,text
0,indiaprogress kinda world amazon web services ...
1,jeffbezos im sir money i mobil phone
2,goals year book aroma success httpstcoijhzlhwchs
3,ola worlds escooter factory sood partners fint
4,amazonin mobile india hmoindia
...,...
3095,amazon gen dot gen dot gen clock price starts rs
3096,deal price discount
3097,deal price discount
3098,videos party india informer


In [11]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people','india','amazon','amazonin','google','indias',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said','trailer','jeffbezos','indi','indian','hi','et','rs',
                 'facebook','tamil','amitagarwal','amitshah','narendramodi','movie','hello','primevideoin','film','video',
                  'prime','videos','watch','music','south','flipkart','book','brand','indie','usa']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

In [13]:
# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aajtak,aamazon,aapl,abhinandanks,ability,abilitythis,abo,abs,abusers,abuses,...,𝒔𝒕𝒓𝒂𝒊𝒏𝒆𝒓,𝒕𝒉𝒂𝒕,𝒕𝒉𝒊𝒔,𝒕𝒐,𝒖𝒔httpstcomucverpscfchefdirect,𝒗𝒆𝒈𝒆𝒕𝒂𝒃𝒍𝒆𝒔httpstcomucverpscf,𝒘𝒂𝒔𝒉,𝒘𝒉𝒆𝒏,𝒘𝒐𝒏𝒕,𝒚𝒐𝒖
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

#corpora.dictionary.Dictionary
# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
id2wordn

{2352: 'indiaprogress',
 2593: 'kinda',
 4802: 'world',
 4739: 'web',
 3970: 'services',
 331: 'azure',
 3212: 'outage',
 324: 'aws',
 4052: 'sir',
 2995: 'money',
 2973: 'mobil',
 3351: 'phone',
 1607: 'goals',
 4834: 'year',
 259: 'aroma',
 4262: 'success',
 1944: 'httpstcoijhzlhwchs',
 3175: 'ola',
 4803: 'worlds',
 1319: 'escooter',
 1382: 'factory',
 4112: 'sood',
 3268: 'partners',
 1452: 'fint',
 2974: 'mobile',
 1783: 'hmoindia',
 4424: 'thewolfpackin',
 2820: 'manufacturing',
 3390: 'plants',
 1038: 'deal',
 3481: 'price',
 1130: 'discount',
 1265: 'element',
 1633: 'gps',
 4089: 'smartwatch',
 2998: 'monitors',
 284: 'aspects',
 317: 'av',
 1968: 'httpstcojnopfpwfqk',
 1621: 'googledown',
 4856: 'youtubers',
 3750: 'retweet',
 200: 'animation',
 2923: 'microsoft',
 532: 'brazil',
 618: 'canada',
 3639: 'reading',
 2253: 'hwsnbn',
 3961: 'series',
 4147: 'spinoff',
 218: 'ap',
 2246: 'huntsman',
 2944: 'mini',
 2239: 'hubs',
 999: 'customs',
 1762: 'hibs',
 3904: 'scimitarss',

### This LDA model only consist on Noun

In [16]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=100)
ldan.print_topics()

[(0,
  '0.044*"price" + 0.042*"deal" + 0.040*"discount" + 0.008*"day" + 0.007*"business" + 0.006*"link" + 0.006*"books" + 0.005*"pmoindia" + 0.005*"world" + 0.004*"face"'),
 (1,
  '0.023*"phones" + 0.019*"amp" + 0.013*"buy" + 0.013*"mobiles" + 0.012*"prices" + 0.011*"cell" + 0.010*"party" + 0.010*"pharmacy" + 0.008*"investment" + 0.007*"amazonhelp"')]

* In Topic modelling there is lots of trail require to find the topic.
* Here we can see the mix of topic, which is difficult to relate.

In [17]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=100)
ldan.print_topics()

[(0,
  '0.029*"amp" + 0.014*"pharmacy" + 0.012*"investment" + 0.010*"day" + 0.010*"business" + 0.007*"life" + 0.007*"eyes" + 0.006*"world" + 0.006*"china" + 0.005*"december"'),
 (1,
  '0.068*"price" + 0.065*"deal" + 0.063*"discount" + 0.039*"phones" + 0.022*"buy" + 0.022*"mobiles" + 0.020*"prices" + 0.020*"cell" + 0.011*"books" + 0.006*"online"'),
 (2,
  '0.016*"party" + 0.009*"feature" + 0.007*"services" + 0.007*"pay" + 0.007*"customer" + 0.007*"amazonhelp" + 0.006*"order" + 0.006*"internet" + 0.006*"fighter" + 0.005*"product"')]

* 1st Topic customer are talking about investment, business in pharma
* 2nd Topic is about product and discount on them
* 3rd Topic is related to after sales service

In [18]:
# Let's try 5 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=5, id2word=id2wordn, passes=200)
ldan.print_topics()

[(0,
  '0.025*"pharmacy" + 0.017*"investment" + 0.014*"books" + 0.012*"pmoindia" + 0.012*"eyes" + 0.010*"online" + 0.009*"fighter" + 0.008*"service" + 0.008*"drrpnishank" + 0.007*"pibindia"'),
 (1,
  '0.060*"phones" + 0.034*"buy" + 0.033*"mobiles" + 0.031*"prices" + 0.030*"cell" + 0.011*"amazonhelp" + 0.010*"pay" + 0.008*"pureminority" + 0.007*"sameekshaindia" + 0.007*"customer"'),
 (2,
  '0.091*"price" + 0.086*"deal" + 0.083*"discount" + 0.043*"amp" + 0.010*"day" + 0.010*"business" + 0.007*"december" + 0.007*"sale" + 0.006*"edition" + 0.006*"face"'),
 (3,
  '0.026*"party" + 0.012*"feature" + 0.010*"china" + 0.007*"internet" + 0.007*"business" + 0.006*"series" + 0.006*"challenge" + 0.005*"customers" + 0.005*"seller" + 0.004*"need"'),
 (4,
  '0.019*"link" + 0.015*"services" + 0.014*"world" + 0.010*"company" + 0.008*"life" + 0.007*"mask" + 0.007*"web" + 0.006*"investment" + 0.006*"startups" + 0.005*"uptake"')]

In [19]:
# Let's try 6 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=6, id2word=id2wordn, passes=200)
ldan.print_topics()

[(0,
  '0.044*"amp" + 0.016*"books" + 0.015*"services" + 0.015*"life" + 0.013*"pay" + 0.010*"world" + 0.010*"pureminority" + 0.009*"mask" + 0.009*"sameekshaindia" + 0.009*"quiz"'),
 (1,
  '0.080*"phones" + 0.045*"buy" + 0.044*"mobiles" + 0.041*"prices" + 0.040*"cell" + 0.008*"gen" + 0.008*"pil" + 0.006*"men" + 0.005*"stories" + 0.005*"bravehearts"'),
 (2,
  '0.031*"party" + 0.030*"pharmacy" + 0.025*"investment" + 0.016*"feature" + 0.015*"amp" + 0.014*"eyes" + 0.008*"chain" + 0.006*"reliance" + 0.005*"users" + 0.004*"home"'),
 (3,
  '0.109*"price" + 0.104*"deal" + 0.100*"discount" + 0.010*"fighter" + 0.008*"link" + 0.007*"china" + 0.006*"battle" + 0.006*"challenge" + 0.006*"air" + 0.006*"support"'),
 (4,
  '0.015*"amazonhelp" + 0.011*"internet" + 0.011*"customer" + 0.010*"day" + 0.008*"series" + 0.007*"order" + 0.006*"service" + 0.006*"brands" + 0.006*"idea" + 0.006*"days"'),
 (5,
  '0.019*"business" + 0.016*"pmoindia" + 0.015*"products" + 0.012*"day" + 0.012*"december" + 0.010*"drrpnis

* 1st Topic - service
* 2nd Topic - mobile phone
* 3rd Topic - investment
* 4th Topic - china
* 5th Topic - customer service
* 6th Topic - Incident happen during december
* But still there is lots fix in topic

### Topic Modeling - Attempt #2 with (Nouns and Adjectives)

In [20]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [21]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.text.apply(nouns_adj))
data_nouns_adj

,text
0,indiaprogress kinda wrong world amazon web ser...
1,jeffbezos im india sir money i mobil phone
2,goals year book sweet aroma success prime http...
3,ola worlds largest escooter factory tamil nadu...
4,amazonin chinese mobile india amazonpmoindia h...
...,...
3095,amazon echo gen dot gen dot gen clock price in...
3096,deal price discount
3097,deal price discount
3098,amazon prime videos party india informer


In [22]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aajtak,aakash,aamazon,aapl,aatmanirbhar,abhigyan,abhinandanks,ability,abilitythis,able,...,𝒔𝒕𝒓𝒂𝒊𝒏𝒆𝒓,𝒕𝒉𝒂𝒕,𝒕𝒉𝒊𝒔,𝒕𝒐,𝒖𝒔httpstcomucverpscfchefdirect,𝒗𝒆𝒈𝒆𝒕𝒂𝒃𝒍𝒆𝒔httpstcomucverpscf,𝒘𝒂𝒔𝒉,𝒘𝒉𝒆𝒏,𝒘𝒐𝒏𝒕,𝒚𝒐𝒖
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [24]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.011*"available" + 0.006*"pmoindia" + 0.006*"services" + 0.006*"amazonhelp" + 0.006*"free" + 0.005*"books" + 0.005*"quiz" + 0.005*"pay" + 0.005*"fighter" + 0.004*"drrpnishank"'),
 (1,
  '0.051*"price" + 0.049*"deal" + 0.047*"discount" + 0.029*"phones" + 0.023*"best" + 0.022*"new" + 0.016*"buy" + 0.016*"mobiles" + 0.015*"prices" + 0.015*"cell"'),
 (2,
  '0.032*"viewed" + 0.026*"amp" + 0.012*"party" + 0.011*"pharmacy" + 0.010*"investment" + 0.008*"potential" + 0.008*"business" + 0.007*"apollo" + 0.006*"feature" + 0.006*"eyes"')]

In [25]:
# Let's try 5 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=200)
ldana.print_topics()

[(0,
  '0.015*"available" + 0.009*"services" + 0.009*"life" + 0.008*"world" + 0.008*"mask" + 0.008*"products" + 0.008*"order" + 0.008*"quiz" + 0.007*"link" + 0.007*"business"'),
 (1,
  '0.009*"pmoindia" + 0.007*"drrpnishank" + 0.007*"party" + 0.006*"pibindia" + 0.006*"internet" + 0.005*"online" + 0.005*"amp" + 0.004*"uninterrupted" + 0.004*"books" + 0.004*"customers"'),
 (2,
  '0.009*"books" + 0.008*"pureminority" + 0.007*"sameekshaindia" + 0.006*"romance" + 0.006*"author" + 0.005*"follow" + 0.005*"online" + 0.005*"anthology" + 0.005*"romancenovels" + 0.005*"ty"'),
 (3,
  '0.052*"viewed" + 0.038*"amp" + 0.019*"pharmacy" + 0.016*"investment" + 0.014*"potential" + 0.013*"party" + 0.012*"apollo" + 0.010*"feature" + 0.009*"eyes" + 0.007*"fighter"'),
 (4,
  '0.073*"price" + 0.070*"deal" + 0.067*"discount" + 0.042*"phones" + 0.034*"new" + 0.030*"best" + 0.024*"buy" + 0.023*"mobiles" + 0.022*"prices" + 0.021*"cell"')]

In [26]:
# Let's try 6 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=6, id2word=id2wordna, passes=200)
ldana.print_topics()

[(0,
  '0.055*"phones" + 0.039*"best" + 0.035*"new" + 0.031*"buy" + 0.030*"mobiles" + 0.029*"prices" + 0.027*"cell" + 0.022*"mobile" + 0.006*"series" + 0.006*"online"'),
 (1,
  '0.013*"pharmacy" + 0.011*"available" + 0.011*"pay" + 0.010*"amazonhelp" + 0.009*"quiz" + 0.008*"investment" + 0.008*"largest" + 0.006*"chain" + 0.006*"order" + 0.006*"loss"'),
 (2,
  '0.011*"pmoindia" + 0.009*"books" + 0.009*"drrpnishank" + 0.008*"pureminority" + 0.007*"sameekshaindia" + 0.007*"romance" + 0.006*"china" + 0.006*"available" + 0.006*"author" + 0.006*"pibindia"'),
 (3,
  '0.079*"price" + 0.076*"deal" + 0.073*"discount" + 0.055*"viewed" + 0.034*"amp" + 0.020*"party" + 0.010*"investment" + 0.010*"feature" + 0.010*"potential" + 0.010*"eyes"'),
 (4,
  '0.012*"world" + 0.010*"life" + 0.009*"mask" + 0.008*"company" + 0.005*"apple" + 0.005*"mibindia" + 0.005*"good" + 0.005*"brands" + 0.005*"growth" + 0.005*"card"'),
 (5,
  '0.019*"business" + 0.013*"services" + 0.012*"small" + 0.010*"day" + 0.009*"interne

### These six topics look pretty decent. Let's settle on these for now.

* Topic 0: Mobile phones which is highest selling
* Topic 1: Amazon service and Investment
* Topic 2: Books and Goverment
* Topic 3: Discount
* Topic 4: Brands Amazon India
* Topic 5: Business and internet services

### These are the Topic customers discuss on social mdeia. Which can give brief understanding of customers and what they like and dislike towards Amazon India product and services. There are near to no negative topic discussion can be seen. This is quit good.

## Flipkart

In [41]:
# read the data in
df1 = pd.read_csv(r"C:\Users\ASUS\abhi\Twitter sentiment analysis\flipkart.csv")

In [42]:
# Let's take a look at the updated text
data_clean1 = pd.DataFrame(df1.text.apply(round1))
data_clean1

,text
0,flipkartsupport just parroted replies with no ...
1,httpstcokzaulfnmyg\n\nmulticolor men amp women...
2,home and bazaar solid wood home temple upto o...
3,flipkart this is thief company
4,indiapoco flipkart poor optimization is softwa...
...,...
3095,flipkartsupport are you trying to say that i s...
3096,it’s a good habit to post honest review of any...
3097,top brands beauty minimum off flipkart\n\n
3098,neerajdark to ensure that your flipkart accoun...


In [43]:
# Let's take a look at the updated text
data_clean1 = pd.DataFrame(data_clean1.text.apply(round2))
data_clean1

,text
0,flipkartsupport just parroted replies with no ...
1,httpstcokzaulfnmygmulticolor men amp women sli...
2,home and bazaar solid wood home temple upto o...
3,flipkart this is thief company
4,indiapoco flipkart poor optimization is softwa...
...,...
3095,flipkartsupport are you trying to say that i s...
3096,its a good habit to post honest review of any ...
3097,top brands beauty minimum off flipkart
3098,neerajdark to ensure that your flipkart accoun...


In [44]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns1 = pd.DataFrame(data_clean1.text.apply(nouns))
data_nouns1

,text
0,flipkartsupport replies substance thats
1,httpstcokzaulfnmygmulticolor men women bagmult...
2,home bazaar wood home temple flipkart
3,flipkart company
4,flipkart optimization software launcherpoco im...
...,...
3095,flipkartsupport i machine service centre
3096,habit review product way consumers
3097,brands flipkart
3098,neerajdark account information please share de...


In [46]:
# Recreate a document-term matrix with only nouns
cvn1 = CountVectorizer(stop_words=stop_words)
data_cvn1 = cvn1.fit_transform(data_nouns1.text)
data_dtmn1 = pd.DataFrame(data_cvn1.toarray(), columns=cvn1.get_feature_names())
data_dtmn1.index = data_nouns1.index
data_dtmn1

,aa,aajtak,aalam,aasimkhan,aatmanirbharbharat,abdomen,abdshah,abhigreeshma,abhishek,abhishekghande,...,ᴾᵒʷᵉʳ,𝐀𝐧𝐬,𝕘⁹,𝕞𝕠𝕥𝕠,𝗕𝘂𝘀𝗶𝗻𝗲𝘀𝘀𝗶𝗻𝗮𝗕𝗼𝘅,𝘽𝙮,𝙇𝙞𝙫𝙚𝙨,𝙎𝙚𝙡𝙡𝙞𝙣𝙜,𝙎𝙤𝙢𝙚𝙩,𝙎𝙤𝙢𝙚𝙩𝙝𝙞𝙣𝙜click
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Create the gensim corpus
corpusn1 = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn1.transpose()))

#corpora.dictionary.Dictionary
# Create the vocabulary dictionary
id2wordn1 = dict((v, k) for k, v in cvn1.vocabulary_.items())

In [48]:
# Let's start with 2 topics
ldan1 = models.LdaModel(corpus=corpusn1, num_topics=5, id2word=id2wordn1, passes=200)
ldan1.print_topics()

[(0,
  '0.069*"flipkartstories" + 0.023*"request" + 0.021*"sorry" + 0.020*"walmart" + 0.020*"deals" + 0.020*"app" + 0.018*"triviatuesdays" + 0.015*"foundation" + 0.015*"concern" + 0.015*"link"'),
 (1,
  '0.021*"day" + 0.020*"flipkartsupport" + 0.014*"sale" + 0.013*"flipkartsellers" + 0.013*"fraud" + 0.012*"hey" + 0.010*"money" + 0.009*"order" + 0.008*"days" + 0.008*"sachinbansal"'),
 (2,
  '0.021*"deal" + 0.018*"delivery" + 0.014*"women" + 0.012*"service" + 0.012*"flipkartshop" + 0.012*"men" + 0.011*"experience" + 0.010*"girls" + 0.008*"belt" + 0.007*"flipkartsupport"'),
 (3,
  '0.059*"account" + 0.049*"flipkartsupport" + 0.039*"information" + 0.035*"product" + 0.032*"order" + 0.015*"details" + 0.014*"service" + 0.014*"message" + 0.012*"days" + 0.012*"id"'),
 (4,
  '0.020*"air" + 0.015*"flipkartsupport" + 0.014*"conditioners" + 0.011*"answers" + 0.009*"play" + 0.009*"party" + 0.007*"pack" + 0.007*"sports" + 0.006*"jagograhakjago" + 0.006*"credit"')]

* 1st Topic - Brand and concern
* 2nd Topic - flipkart serivce and fraud
* 3rd Topic - Brand experience
* 4th Topic - complaint
* 5th Topic - product and games offer on flipkart
* But still there is lots fix in topic

### Topic Modeling - Attempt #2 (Nouns and Adjectives)

In [49]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj1 = pd.DataFrame(data_clean1.text.apply(nouns_adj))
data_nouns_adj1

,text
0,flipkartsupport replies substance thats
1,httpstcokzaulfnmygmulticolor men women bagmult...
2,home bazaar solid wood home temple flipkart
3,flipkart thief company
4,indiapoco flipkart poor optimization software ...
...,...
3095,flipkartsupport i machine service centre
3096,good habit honest review product best way cons...
3097,top brands beauty minimum flipkart
3098,neerajdark flipkart account information safe p...


In [50]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna1 = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna1 = cvna1.fit_transform(data_nouns_adj1.text)
data_dtmna1 = pd.DataFrame(data_cvna1.toarray(), columns=cvna1.get_feature_names())
data_dtmna1.index = data_nouns_adj1.index
data_dtmna1

,aa,aajtak,aalam,aamchi,aasimkhan,aatmanirbharbharat,abdomen,abdshah,abhigreeshma,abhishek,...,𝐀𝐧𝐬,𝕘⁹,𝕞𝕠𝕥𝕠,𝗕𝘂𝘀𝗶𝗻𝗲𝘀𝘀𝗶𝗻𝗮𝗕𝗼𝘅,𝘽𝙮,𝙀𝙫𝙚𝙧𝙮𝙗𝙤𝙙𝙮,𝙇𝙞𝙫𝙚𝙨,𝙎𝙚𝙡𝙡𝙞𝙣𝙜,𝙎𝙤𝙢𝙚𝙩,𝙎𝙤𝙢𝙚𝙩𝙝𝙞𝙣𝙜click
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# Create the gensim corpus
corpusna1 = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna1.transpose()))

# Create the vocabulary dictionary
id2wordna1 = dict((v, k) for k, v in cvna1.vocabulary_.items())

In [52]:
# Let's try 3 topics
ldana1 = models.LdaModel(corpus=corpusna1, num_topics=5, id2word=id2wordna1, passes=200)
ldana1.print_topics()

[(0,
  '0.015*"sorry" + 0.013*"flipkartsupport" + 0.010*"casual" + 0.010*"concern" + 0.010*"micromaxindia" + 0.010*"men" + 0.009*"women" + 0.009*"team" + 0.008*"answers" + 0.008*"sale"'),
 (1,
  '0.021*"days" + 0.021*"fake" + 0.011*"mi" + 0.010*"big" + 0.010*"tv" + 0.008*"xiaomiindia" + 0.008*"product" + 0.008*"services" + 0.007*"answers" + 0.007*"saving"'),
 (2,
  '0.049*"flipkartsupport" + 0.036*"order" + 0.025*"product" + 0.018*"service" + 0.014*"delivery" + 0.012*"refund" + 0.011*"app" + 0.011*"mobile" + 0.011*"deals" + 0.011*"customer"'),
 (3,
  '0.052*"flipkartstories" + 0.034*"flipkartsupport" + 0.020*"walmart" + 0.014*"air" + 0.014*"triviatuesdays" + 0.013*"nokia" + 0.011*"foundation" + 0.010*"conditioners" + 0.008*"hey" + 0.007*"kalyank"'),
 (4,
  '0.067*"account" + 0.044*"information" + 0.044*"safe" + 0.019*"private" + 0.017*"details" + 0.016*"message" + 0.013*"share" + 0.010*"request" + 0.010*"sure" + 0.009*"link"')]

### These five topics look pretty decent. Let's settle on these for now.

* Topic 0: flipkart service and micromax
* Topic 1: mi, tv, xiaomi
* Topic 2: Sales service and after sales service
* Topic 3: flipkart and nokia
* Topic 4: account privacy

### These are the Topic customers discuss on social mdeia. Which can give brief understanding of customers and what they like and dislike towards Flipkart product and services. There are negative topic seen in discussion. Which is quit worrying. 